In [ ]:
Another difference is that the list.sort() method is only defined for lists. In contrast, 
the sorted() function accepts any iterable.

In [14]:
# Why do lambdas defined in a loop with different values all return the same result?¶

squares = []
for x in range(5):
    squares.append(lambda: x**2)
    
squares         #This gives you a list that contains 5 lambdas that calculate x**2. You might expect that, when called, they would return, respectively, 0, 1, 4, 9, and 16. However, when you actually try you will see that they all return 16:

[<function __main__.<lambda>()>,
 <function __main__.<lambda>()>,
 <function __main__.<lambda>()>,
 <function __main__.<lambda>()>,
 <function __main__.<lambda>()>]

In [12]:
squares[2]()

16

In [13]:
squares[4]()

16

In [17]:
# This happens because x is not local to the lambdas, but is defined in the outer scope, and it is accessed when the 
# lambda is called — not when it is defined. At the end of the loop, the value of x is 4, so all the functions now 
# return 4**2, i.e. 16. You can also verify this by changing the value of x and see how the results of the lambdas change:


# In order to avoid this, you need to save the values in variables local to the lambdas, so that they don’t rely on 
# the value of the global x:
    
squares = []
for x in range(5):
    squares.append(lambda n=x: n**2)

# Here, n=x creates a new variable n local to the lambda and computed when the lambda is defined so that it has the same
# value that x had at that point in the loop. This means that the value of n will be 0 in the first lambda, 1 in the second,
# 2 in the third, and so on. Therefore each lambda will now return the correct result:

print(squares[2]())
squares[4]()


#Note that this behaviour is not peculiar to lambdas, but applies to regular functions too.

4


16

In [ ]:
# How do I share global variables across modules?

# The canonical way to share information across modules within a single program is to create a special module 
# (often called config or cfg). Just import the config module in all modules of your application; the module then becomes
# available as a global name. Because there is only one instance of each module, any changes made to the module object get
# reflected everywhere.

config.py:

x = 0   # Default value of the 'x' configuration setting

mod.py:

import config
config.x = 1

main.py:

import config
import mod
print(config.x)


# Note that using a module is also the basis for implementing the singleton design pattern, for the same reason.

In [26]:
# Why are default values shared between objects in Python?

def foo(mydict={}):
    ... calculate...
    mydict[key] = value
    return mydict

We have a function. In that we have a Dictionary as a parameter with a default value. The first time you call this function,
mydict contains a single item. The second time, mydict contains two items because when foo() begins executing, 
mydict starts out with an item already in it.

We often expect that a function call creates new objects for default values. However, this is not the case.
The Default values are created exactly once, when the function is defined. If that object is changed, like the dictionary
in the above example, the subsequent calls to the function will refer to this changed object.


# The default values concept in Python are based on using mutable or immutable objects. It is good programming practice
# to not use mutable objects as default values. Instead, use None as the default value to avoid issues. 

# Immutable objects such as numbers, strings, tuples, and None, are safe from change.
# Changes to mutable objects such as dictionaries, lists, and class instances can lead to confusion.

def foo(mydict=None):
    if mydict is None:
        mydict = {}  # create a new dict for local namespace
        
        
        
This feature can be useful. When you have a function that’s time-consuming to compute, a common technique is to cache the parameters and the resulting value of each call to the function, and return the cached value if the same value is requested again. This is called “memoizing”, and can be implemented like this:

# Callers can only provide two parameters and optionally pass _cache by keyword
def expensive(arg1, arg2, *, _cache={}):
    if (arg1, arg2) in _cache:
        return _cache[(arg1, arg2)]

    # Calculate the value
    result = ... expensive computation ...
    _cache[(arg1, arg2)] = result           # Store result in the cache
    return result

In [27]:
# How can I pass optional or keyword parameters from one function to another?

# Collect the arguments using the * and ** specifiers in the function’s parameter list; this gives you the positional arguments as a tuple and the keyword arguments as a dictionary. You can then pass these arguments when calling another function by using * and **:

def f(x, *args, **kwargs):
    ...
    kwargs['width'] = '14.3c'
    ...
    g(x, *args, **kwargs)

In [ ]:
# What is the difference between arguments and parameters?

# Parameters are defined by the names that appear in a function definition, whereas arguments are the values actually passed to a function when calling it. Parameters define what kind of arguments a function can accept. For example, given the function definition:

def func(foo, bar=None, **kwargs):
    pass

# foo, bar and kwargs are parameters of func. However, when calling func, for example:

func(42, bar=314, extra=somevar)

# the values 42, 314, and somevar are arguments.


In [33]:
# Why did changing list ‘y’ also change list ‘x’?

x = []
y = x
y.append(10)
print(y)
print(x)

# Beacuse of two factors that produce this result:

# 1. Variables are simply names that refer to objects. Doing y = x doesn’t create a copy of the list – it creates a new
# variable y that refers to the same object x refers to. This means that there is only one object (the list), and 
# both x and y refer to it.

# 2. Lists are mutable, which means that you can change their content. After the call to append(), the content of the 
# mutable object has changed from [] to [10]. Since both the variables refer to the same object, using either name 
# accesses the modified value [10].


# If we instead assign an immutable object to x:
x = 5  # ints are immutable
y = x
x = x + 1  # 5 can't be mutated, we are creating a new object here

print(y)
print(x)  # This is because integers are immutable, 

# Some operations (for example y.append(10) and y.sort()) mutate the object, whereas superficially similar operations 
# (for example y = y + [10] and sorted(y)) create a new object.In general in Python (and in all cases in the standard library)
# a method that mutates an object will return None to help avoid getting the two types of operations confused.

[10]
[10]
5
6


In [ ]:
If you want to know if two variables refer to the same object or not, you can use the is operator,
or the built-in function id().

In [ ]:
# How do you make a higher order function in Python?

# You have two choices: you can use nested scopes or you can use callable objects. For example, 
# suppose you wanted to define linear(a,b) which returns a function f(x) that computes the value a*x+b. Using nested scopes:

def linear(a, b):
    def result(x):
        return a * x + b
    return result

# Or using a callable object:

class linear:

    def __init__(self, a, b):
        self.a, self.b = a, b

    def __call__(self, x):
        return self.a * x + self.b
    
# In both cases,

taxes = linear(0.3, 2)


# The callable object approach has the disadvantage that it is a bit slower and results in slightly longer code. 
# However, note that a collection of callables can share their signature via inheritance:

class exponential(linear):
    # __init__ inherited
    def __call__(self, x):
        return self.a * (x ** self.b)
    
# Object can encapsulate state for several methods:

class counter:

    value = 0

    def set(self, x):
        self.value = x

    def up(self):
        self.value = self.value + 1

    def down(self):
        self.value = self.value - 1

count = counter()
inc, dec, reset = count.up, count.down, count.set

# Here inc(), dec() and reset() act like functions which share the same counting variable.

In [ ]:
# How do I copy an object in Python?

# In general, try copy.copy() or copy.deepcopy() for the general case. Not all objects can be copied, but most can.

# Some objects can be copied more easily. Dictionaries have a copy() method:

newdict = olddict.copy()
# Sequences can be copied by slicing:

new_l = l[:]

In [ ]:
How can I find the methods or attributes of an object?  dir(x) 

True

In [ ]:
How do I specify hexadecimal and octal integers?

0o10   or  0O10      # octal value with a zero, and then a lower or uppercase “o”.

a = 0xa5             # precede the hexadecimal number with a zero, and then a lower or uppercase “x”.
b = 0XB2

2
-3
2
-1


True

In [41]:
int('144')
float('144')

# Do not use the built-in function eval() if all you need is to convert strings to numbers. 
# eval() will be significantly slower and it presents a security risk:
# eval() also has the effect of interpreting numbers as Python expressions, 
# so that e.g. eval('09') gives a syntax error because Python does not allow leading ‘0’ in a decimal number (except ‘0’).


# For fancy formatting, see the f-strings and Format String Syntax sections, 
"{:04d}".format(144)      # yields '0144' 
"{:.3f}".format(1.0/3.0)  # yields '0.333'.

'0.333'

In [3]:
# How do I modify a string in place?

# You can’t, because strings are immutable. In most situations, you should simply construct a new string from 
# the various parts you want to assemble it from. However, if you need an object with the ability to modify in-place
# unicode data, try using an io.StringIO object or the array module:

import io
s = "Hello, world All"
sio = io.StringIO(s)
print(sio.read())      #Reading file using StringIO
print(sio.getvalue())
print(sio.seek(7))
print(sio.write("there!"))
print(sio.getvalue())




import array
a = array.array('u', s)
print(a)

a[0] = 'y'
print(a)

a.tounicode()

Hello, world All
Hello, world All
7
6
Hello, there!All
array('u', 'Hello, world All')
array('u', 'yello, world All')


'yello, world All'

In [13]:
# A raw string ending with an odd number of backslashes will escape the string’s quote:

r'C:\this\will\not\work\'

SyntaxError: unterminated string literal (detected at line 3) (410726696.py, line 3)

In [19]:
'C:\\this\\will\\work\\'     # One is to use regular strings and double the backslashes:
r'C:\this\will\work' '\\'    #Another is to concatenate a regular string containing an escaped backslash to the raw string:
import os
os.path.join(r'C:\this\will\work', '')     # It is also possible to use os.path.join() to append a backslash on Windows:

r'backslash\'preserved'    #while a backslash will “escape” a quote for the purposes of determining where the raw string ends, no escaping occurs when interpreting the value of the raw string. That is, the backslash remains present in the value of the raw string:

"backslash\\'preserved"